# Publish PlanetScope Imagery to ArcGIS Online

These notebooks contain samples for publishing PlanetScope imagery as image services in ArcGIS Image for ArcGIS Online.  With Planet imagery published in ArcGIS Online, you are able to:

* Use the imagery in analytics workflows using raster functions or raster analytics
* Access full bit-depth imagery for custom stretching or band combinations performed on the fly
* Securely share imagery with your end-users using the ArcGIS identity, named users, in ArcGIS Online

This script specifically works with PlanetScope 8-band analytics surface reflectance assets, but could be modified to work with additional asset types.  For example, this could be extended to support Planet Basemaps or SkySat imagery.

Want to see an example of the output of this notebook now? [Check out this map here.](https://planetlabs.maps.arcgis.com/apps/instant/basic/index.html?appid=eaeaa5cf78694c63b9955ff7c87e479f)

### Prerequisites

* Access to ArcGIS Online with a proper privileges to [host imagery in ArcGIS Online](https://doc.arcgis.com/en/imagery/workflows/resources/hosting-imagery-with-arcgis-image-for-arcgis-online.htm)
* A previously placed order for PlanetScope imagery, either through our [Orders API](https://docs.planet.com/develop/apis/orders), [Planet Add-in for ArcGIS Pro](https://docs.planet.com/platform/integrations/arcgis/), or [Explorer](https://www.planet.com/explorer)


In [ ]:
# Install ArcGIS package if not already installed
%pip install arcgis

In [2]:
# Import required libraries

import arcgis
from arcgis.raster.analytics import create_image_collection
import os
import planet 
import asyncio
import glob
from zipfile import ZipFile
from pathlib import Path
import pandas as pd
import datetime

In [3]:
# Connect to Planet API

# If you are not already logged in, this will prompt you to open a web browser to log in.
auth = planet.Auth.from_profile('planet-user', save_state_to_storage=True)
if not auth.is_initialized():
    auth.user_login(allow_open_browser=False, allow_tty_prompt=True)

session = planet.Session(auth)
pl = planet.Planet(session)

First, you need to provide an order ID to publish.  You could get the order ID from:

* Manually from [Planet Explorer](https://www.planet.com/explorer) or your [account orders page](https://insights.planet.com/data/orders)
* Programatically from the Orders API

For example, a script could be used to search for orders from with the last hour to then be published to ArcGIS Online.

In [4]:
# Collect all orders
all_orders = []

# Get the generator from list_orders
orders_generator = pl.orders.list_orders()

# Iterate through the generator to get each order
for order in orders_generator:
    all_orders.append(order)

# Extract relevant fields into a list of dictionaries
orders_data = []
for order in all_orders:
    # Extract product details if available
    item_type = "N/A"
    product_bundle = "N/A"
    if order.get('products') and len(order['products']) > 0:
        item_type = order['products'][0].get('item_type', 'N/A')
        product_bundle = order['products'][0].get('product_bundle', 'N/A')
    
    # Create a simplified dictionary with just the fields we want
    order_info = {
        'ID': order.get('id', 'N/A'),
        'Created': order.get('created_on', 'N/A'),
        'Name': order.get('name', 'N/A'),
        'State': order.get('state', 'N/A'),
        'Item Type': item_type,
        'Product Bundle': product_bundle
    }
    orders_data.append(order_info)

# Create and display DataFrame
orders_df = pd.DataFrame(orders_data)
orders_df

,ID,Created,Name,State,Item Type,Product Bundle
0,44656e61-87b6-4b88-a19a-aabe9a65394b,2025-09-24T20:53:32.952408Z,San Antonio PlanetScope - Premium Quality - 20...,success,PSScene,analytic_udm2
1,ad7ad99d-689a-43b6-b7a8-a63589b06d76,2025-09-24T20:50:38.436739Z,San Antonio PlanetScope - Premium Quality - 20...,success,PSScene,analytic_udm2
2,f728a96c-6bbd-43c3-8965-b94cbd634187,2025-09-24T20:34:13.861486Z,San Antonio Clear Sky Imagery - Sept 2025,success,PSScene,analytic_udm2
3,032a5e00-7e40-46af-ad43-0ca9dded211b,2025-07-31T20:44:06.476425Z,test 2,success,SkySatScene,basic_analytic_udm2
4,cafdae84-8bcc-4783-a58a-d7d470e0dce5,2025-07-31T20:21:24.984519Z,test sky sat scenes,success,SkySatScene,analytic_sr_udm2


In [ ]:
# Select the first order ID to use in the next steps
my_order_id = orders_df.iloc[0].ID
# # Or specify your own order ID below
# my_order_id = "insert-your-order-id-here"

my_order_name = pl.orders.get_order(my_order_id)['name']

print(f'Order to be published: {my_order_name} - id: {my_order_id}')

Order to be published: San Antonio Clear Sky Imagery - Sept 2025 - id: f728a96c-6bbd-43c3-8965-b94cbd634187


Now that we have an order, we are going to download it. Planet also supports cloud delivery and the option to store you data on the platform natively, but here we will download locally because we are going to be uploading the imagery to ArcGIS Online which is a software-as-a-service that does not read directly from cloud storage. Other ArcGIS imagery solutions can read directly from cloud storage if that is required.  

In [ ]:
# Get current working directory
cwd = os.getcwd()

# Create download directory if it doesn't exist
download_dir = Path(cwd) / "planet_downloads"
download_dir.mkdir(exist_ok=True)

# Create order-specific directory
order_dir = download_dir / my_order_id
order_dir.mkdir(exist_ok=True)

# Array to store local file paths
local_tiff_files = []

# Get the order details and name
order_details = pl.orders.get_order(my_order_id)
order_name = order_details['name']
print(f"Processing order: {order_name}")
print(f"All files will be stored in: {order_dir}")

# Check if the order has been successfully completed
if order_details['state'] != 'success':
    print("Order isn't completed yet")
    raise Exception("Order not ready for download")

# Check if the order contains zip archives or direct file links
zip_archives = [r['name'] for r in order_details['_links']['results'] if r['name'].endswith(".zip")]

if len(zip_archives) > 0:
    print("Order contains zip archives - downloading and extracting...")
    
    # Create subfolders for organization
    raw_dir = order_dir / "raw_downloads"
    extracted_dir = order_dir / "extracted"
    raw_dir.mkdir(exist_ok=True)
    extracted_dir.mkdir(exist_ok=True)
    
    # Download all assets including zip archives to the raw downloads folder within order directory
    download_results = pl.orders.download_order(
        my_order_id, 
        directory=str(raw_dir),  # Download to raw folder within order directory
        overwrite=True,
        progress_bar=True
    )
    
    # Create a list of all zip files that were downloaded
    zip_files = [Path(x) for x in download_results if Path(x).suffix == ".zip"]
    
    # Extract each zip file to the extracted folder
    for zip_file in zip_files:
        print(f"Extracting {zip_file.name}...")
        with ZipFile(zip_file) as z:
            z.extractall(extracted_dir)  # Extract to extracted folder
    
    # Find all relevant tiff files in the extracted directory
    local_tiff_files = [file for file in extracted_dir.glob("**/*.tif") if "udm" not in file.name.lower()]

else:
    print("Order contains direct file links - downloading individual files...")
    
    # Download each tiff file individually to the order directory
    for result in order_details['_links']['results']:
        filename = result['name']
        location = result['location']
        
        print(f"Downloading {filename}...")
        downloaded_file = pl.orders.download_asset(
            location=location,
            filename=filename,
            directory=str(order_dir),  # Download directly to order directory
            overwrite=True,
            progress_bar=True
        )
        
    # Find all relevant tiff files in the order directory
    local_tiff_files = [file for file in order_dir.glob("**/*.tif") if "udm" not in file.name.lower()]
    
print(f"\nDownload complete! Found {len(local_tiff_files)} SR tiff files:")
for file_path in local_tiff_files:
    print(f"  {file_path}")

print(f"\nAll files stored in: {order_dir}")
print(f"File paths available in 'local_tiff_files' array")

# The local_tiff_files array now contains Path objects for all downloaded tiff files
# You can use these for processing and delete them later when done

## Publish to ArcGIS Online

Now the imagery can be published to ArcGIS Online!  Simply authenticate to ArcGIS Online, create a unique name for your imagery layer, and publish the imagery layer.

In [ ]:
# Connect to ArcGIS Online
gis = arcgis.gis.GIS(url="https://www.arcgis.com", client_id="your-arcgis-client-id")
gis

In [ ]:
if not local_tiff_files:
    print("No tiff files found. Make sure to run the Planet download code first.")
else:
    # Create a unique timestamp
    timestamp = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
    
    # Use timestamp and order name to create a unique name for the image collection
    # Extract a meaningful identifier from the first file name
    first_file_stem = local_tiff_files[0].stem
    scene_id = first_file_stem.split('_')[0]  # Get the scene ID part
    layer_name = f"PlanetLabs_{scene_id}_{timestamp}"
    
    print(f"Creating image collection: {layer_name}")
    print(f"Processing {len(local_tiff_files)} Planet images...")
    
    # Convert Path objects to strings for the API
    # Use the order directory directly since everything is now flat
    order_dir = download_dir / my_order_id
    input_folder = str(order_dir)  # Use the order-specific folder containing all images
    
    # Create the image collection using Planet imagery
    # Note: This can take significant time with large datasets
    try:
        published_imagery_layer = create_image_collection(
            image_collection=layer_name,
            input_rasters=input_folder,
            raster_type_name="Raster Dataset",  # Generic raster type - ArcGIS will detect metadata
            context={
                "outSR": {"wkid": 3857},  # Web Mercator projection
                "defineNodata": True,
                "noDataArguments": {
                    "noDataValues": [0],  # Planet uses 0 for nodata
                    "compositeValue": True
                },
                "buildFootprints": True,  # Enable footprints for better mosaicking
                "buildOverview": True,
                "image_collection_properties": {
                    "imageCollectionType": "Satellite"
                }
            },
            gis=gis
        )
        
        print(f"✅ Successfully created image collection: {layer_name}")
        print(f"📍 Image service URL: {published_imagery_layer.url}")
        print(f"🆔 Item ID: {published_imagery_layer.itemid}")
        
        # Store the result for later use
        planet_image_collection = published_imagery_layer
        
    except Exception as e:
        print(f"❌ Error creating image collection: {str(e)}")
        print("This might be due to:")
        print("- Insufficient ArcGIS Online privileges")
        print("- Raster analysis not configured properly")
        print("- Large dataset size (try with fewer images first)")

## View the New Image Service and Clean Up Folders
Now we can view the Image Service by drawing it on a map directly in this notebook!  Or you can view it in your ArcGIS Online environment.

In [ ]:
# view the new imagery layer on an arcgis map
# this layer can now be added to other maps, analyzed with ArcGIS Raster Analytics tools, and more

my_map = gis.map(location = planet_image_collection.extent, zoomlevel = 11)
my_map.basemap = "imagery"
my_map.add_layer(planet_image_collection)
my_map